In [ ]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import sqlite3
import tkinter as tk

def scrape_and_save_data():
    # Connect to the database
    conn = sqlite3.connect('movie_data1.db')
    c = conn.cursor()

    # Create the table if it doesn't exist
    c.execute('''CREATE TABLE IF NOT EXISTS top_rated_movies 
                 (rank INTEGER, title TEXT, year INTEGER, ratings REAL)''')

    # Scrape the data
    url = 'https://www.imdb.com/chart/top/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    info = soup.find('tbody', class_='lister-list').find_all('tr')

    # Insert the data into the database and save to excel
    excel = openpyxl.Workbook()
    sheet = excel.active
    sheet.title = 'top rated movies'

    sheet.append(['Movie rank', 'Movie name', 'Year of release', 'Ratings'])

    for i in info:
        title = i.find('td', class_='titleColumn').a.text
        ratings = i.find('td', class_='ratingColumn').strong.text
        rank = i.find('td', class_='titleColumn').get_text(strip=True).split(".")[0]
        year = i.find('td', class_='titleColumn').span.text.strip('()')
        
        sheet.append([rank, title, year, ratings])
        c.execute("INSERT INTO top_rated_movies (rank, title, year, ratings) VALUES (?, ?, ?, ?)", 
                  (rank, title, year, ratings))

    # Commit the changes to the database and close the connection
    conn.commit()
    conn.close()

    # Save the excel file
    excel.save('movie data1.xlsx')
    result_label.configure(text="Data scraped and saved to database and Excel file.")

# Create the GUI window
window = tk.Tk()
window.title("IMDB Data Scraper")

# Create the widgets
title_label = tk.Label(window, text="IMDB Data Scraper", font=("Arial", 18))
desc_label = tk.Label(window, text="Click the button to scrape data from IMDB and save it to a database and Excel file.")
button = tk.Button(window, text="Scrape and Save Data", command=scrape_and_save_data)
result_label = tk.Label(window, text="")

# Add the widgets to the window
title_label.pack(pady=10)
desc_label.pack(pady=10)
button.pack(pady=10)
result_label.pack(pady=10)

# Start the GUI event loop
window.mainloop()
